In [218]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report
import seaborn as sns

import warnings 
warnings.filterwarnings('ignore')

In [219]:
df = sns.load_dataset('iris')

In [220]:
X = df.drop('species', axis=1)
y = df['species']

In [221]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

### train test split needed to have the complete unknown data on which we can test later

## Stacking

In [222]:
base_learners = [
    ('dt', DecisionTreeClassifier(random_state=42)),
    ('svc', SVC(probability=True, kernel='rbf', random_state=42)),
    ('lr', LogisticRegression(max_iter=1000)),
]

In [223]:
meta_learner = LogisticRegression(max_iter=1000)

In [224]:
stacking_clf = StackingClassifier(
    estimators=base_learners,
    final_estimator=meta_learner,
    cv = 5    # cross validation on X train data unlike grid seach cv
)

In [225]:
stacking_clf.fit(X_train, y_train)

StackingClassifier(cv=5,
                   estimators=[('dt', DecisionTreeClassifier(random_state=42)),
                               ('svc', SVC(probability=True, random_state=42)),
                               ('lr', LogisticRegression(max_iter=1000))],
                   final_estimator=LogisticRegression(max_iter=1000))

In [226]:
y_pred = stacking_clf.predict(X_test)

In [227]:
accuracy_score(y_test, y_pred)

0.9666666666666667

## Bagging (Random Forest)

In [228]:
from sklearn.ensemble import RandomForestClassifier

In [229]:
rf_model = RandomForestClassifier(
    n_estimators=100, # no of trees
    max_depth=None,  # let trees grow
    random_state=42
)

In [230]:
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [231]:
y_pred = rf_model.predict(X_test)

In [232]:
accuracy_score(y_test, y_pred)

0.9

## Boosting

In [233]:
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier

from xgboost import XGBClassifier

### Adaboost

In [234]:
ada_model = AdaBoostClassifier(n_estimators=100, random_state=42)

In [235]:
ada_model.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100, random_state=42)

In [236]:
y_pred = ada_model.predict(X_test)

In [237]:
accuracy_score(y_test,y_pred)

0.9

### Gradient Boost

In [238]:
gb_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, random_state=42)

In [239]:
gb_model.fit(X_train,y_train)

GradientBoostingClassifier(random_state=42)

In [240]:
y_pred = gb_model.predict(X_test)

In [241]:
accuracy_score(y_test, y_pred)

0.9666666666666667

### XGBOOST

In [242]:
xgb_model = XGBClassifier(n_estimators=100, learning_rate=0.1, random_state=42, max_depth=3, use_label_encoder=False, eval_metric='mlogloss')

only need encoded data

In [243]:
from sklearn.preprocessing import LabelEncoder

# Encode target labels
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [244]:
xgb_model.fit(X_train, y_train_enc)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, ...)

In [245]:
y_pred = xgb_model.predict(X_test)

In [246]:
accuracy_score(y_test_enc, y_pred)

0.9333333333333333